In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import random
import difflib
from IPython.display import display, HTML
from Levenshtein import distance as levenshtein_distance

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# 1. SpellGram 데이터셋 다운

In [10]:
print("Downloading SpellGram dataset...")
ds = load_dataset("vishnun/SpellGram")

print("\nDownload Complete!")


Download Complete!


In [28]:
print("\n=== Sample Data (First 3 examples) ===")
# Pandas로 변환하여 보기 좋게 출력
df_sample = pd.DataFrame(ds[first_split].select(range(3)))
df_sample


=== Sample Data (First 3 examples) ===


,source,target,error_type
0,rate the silent upeaker four out oe 6,rate the silent speaker four out of 6,다중 오류
1,please find me tqe gork tqe bfrning sorld,please find me the work the burning world,다중 오류
2,three friendl afe relaxing uround the tsble,three friends are relaxing around the table,다중 오류


# 2. 데이터 구조 파악

## 데이터셋 구조, 피쳐 이름, 사이즈

In [29]:
print("\n=== Dataset Structure ===")
print(ds)

print("\n=== Feature Names ===")
first_split = list(ds.keys())[0]
print(f"Split: {first_split}")
print(ds[first_split].features)

print("\n=== Dataset Sizes ===")
for split in ds.keys():
    print(f"{split}: {len(ds[split])} rows")


=== Dataset Structure ===
DatasetDict({
    train: Dataset({
        features: ['source', 'target', 'error_type'],
        num_rows: 40000
    })
})

=== Feature Names ===
Split: train
{'source': Value('string'), 'target': Value('string'), 'error_type': Value('string')}

=== Dataset Sizes ===
train: 40000 rows


## 문장 길이 통계

In [20]:
print("\n" + "=" * 60)
print("문장 길이 통계")
print("=" * 60)

train_data = ds["train"]

# 길이 리스트 계산 (source / target)
src_lengths = []
tgt_lengths = []

for ex in train_data:
    src = ex.get("source")
    tgt = ex.get("target")

    if src is not None:
        src_lengths.append(len(src.split()))

    if tgt is not None:
        tgt_lengths.append(len(tgt.split()))

# ─────────────────────────────
# 통계 출력
# ─────────────────────────────
print("\nSource (오타 문장):")
print(f"  - Min: {min(src_lengths)}")
print(f"  - Max: {max(src_lengths)}")
print(f"  - Mean: {np.mean(src_lengths):.2f}")
print(f"  - Median: {np.median(src_lengths):.2f}")

print("\nTarget (정정 문장):")
print(f"  - Min: {min(tgt_lengths)}")
print(f"  - Max: {max(tgt_lengths)}")
print(f"  - Mean: {np.mean(tgt_lengths):.2f}")
print(f"  - Median: {np.median(tgt_lengths):.2f}")

# ─────────────────────────────
# 길이 분포
# ─────────────────────────────
print("\nLength distribution (Source 기준):")
thresholds = [10, 20, 30, 40, 50]

for th in thresholds:
    count = sum(l <= th for l in src_lengths)
    percentage = (count / len(src_lengths)) * 100
    print(f"  - {th} 단어 이하: {percentage:.1f}%")


문장 길이 통계

Source (오타 문장):
  - Min: 1
  - Max: 40
  - Mean: 9.78
  - Median: 10.00

Target (정정 문장):
  - Min: 1
  - Max: 40
  - Mean: 9.78
  - Median: 10.00

Length distribution (Source 기준):
  - 10 단어 이하: 56.9%
  - 20 단어 이하: 99.8%
  - 30 단어 이하: 100.0%
  - 40 단어 이하: 100.0%
  - 50 단어 이하: 100.0%


## 오류 유형 계산

In [30]:
# 오류 유형 계산 함수
def calculate_error_type(example):
    source = example.get("source")
    target = example.get("target")

    # None 또는 결측 처리
    if source is None or target is None:
        return {"error_type": "데이터 없음"}

    src_words = str(source).split()
    tgt_words = str(target).split()

    # 단어 개수 차이
    if len(src_words) != len(tgt_words):
        return {"error_type": "단어 추가/삭제"}

    # 동일한 길이일 때 단어 단위 비교
    diff_count = sum(1 for s, t in zip(src_words, tgt_words) if s != t)

    if diff_count == 1:
        return {"error_type": "단일 단어 오타"}
    elif diff_count > 1:
        return {"error_type": "다중 오류"}
    else:
        return {"error_type": "변경 없음"}  # 혹시 source와 target이 같은 경우


# map 적용 (progress bar 표시됨)
for split in ds:
    print(f"Processing split: {split}")
    ds[split] = ds[split].map(calculate_error_type)

Processing split: train


Map: 100%|██████████| 40000/40000 [00:00<00:00, 49320.73 examples/s]


In [31]:
pd.DataFrame(ds["train"][:3])

,source,target,error_type
0,rate the silent upeaker four out oe 6,rate the silent speaker four out of 6,다중 오류
1,please find me tqe gork tqe bfrning sorld,please find me the work the burning world,다중 오류
2,three friendl afe relaxing uround the tsble,three friends are relaxing around the table,다중 오류


In [33]:
df = ds["train"].to_pandas()

# error_type 목록
error_types = df["error_type"].unique()

print("=" * 80)
print("🔍 오류 유형별 예시 문장")
print("=" * 80)

for et in error_types:
    subset = df[df["error_type"] == et]

    print(f"\n=== 오류 유형: {et} ===")
    print(f"총 {len(subset)}개 중 예시 3개\n")

    for idx, row in subset.head(3).iterrows():
        print(f"- Source : {row['source']}")
        print(f"- Target : {row['target']}")
        print("")

🔍 오류 유형별 예시 문장

=== 오류 유형: 다중 오류 ===
총 28514개 중 예시 3개

- Source : rate the silent upeaker four out oe 6
- Target : rate the silent speaker four out of 6

- Source : please find me tqe gork tqe bfrning sorld
- Target : please find me the work the burning world

- Source : three friendl afe relaxing uround the tsble
- Target : three friends are relaxing around the table


=== 오류 유형: 단일 단어 오타 ===
총 11042개 중 예시 3개

- Source : what dm they want
- Target : what do they want

- Source : man in tan aat working with stones
- Target : man in tan hat working with stones

- Source : he was one of the main reasons for our qucceqq
- Target : he was one of the main reasons for our success


=== 오류 유형: 변경 없음 ===
총 442개 중 예시 3개

- Source : nearby southend airport started life as a grass fighter station in world war one
- Target : nearby southend airport started life as a grass fighter station in world war one

- Source : the village name hitcham is anglo saxon in origin and means hycgas homestead
- Tar

In [34]:
ds["train"].to_pandas()["error_type"].value_counts(normalize=True)

error_type
다중 오류       0.712850
단일 단어 오타    0.276050
변경 없음       0.011050
단어 추가/삭제    0.000025
데이터 없음      0.000025
Name: proportion, dtype: float64

## 더 자세한 샘플 보기

In [46]:
print("\n" + "=" * 60)
print("📖 랜덤 상세 샘플 + 단어 하이라이트 (10개)")
print("=" * 60)

# 랜덤 인덱스 10개 선택
indices = random.sample(range(len(train_data)), 10)

def highlight_diff(src_text, tgt_text):
    """source와 target을 단어 단위로 비교해서 다른 부분만 강조"""
    src_words = src_text.split()
    tgt_words = tgt_text.split()

    highlighted_src = []
    highlighted_tgt = []

    for s, t in zip(src_words, tgt_words):
        if s != t:
            highlighted_src.append(f"<b>{s}</b>")
            highlighted_tgt.append(f"<b>{t}</b>")
        else:
            highlighted_src.append(s)
            highlighted_tgt.append(t)

    return " ".join(highlighted_src), " ".join(highlighted_tgt)


html_output = ""

for i, idx in enumerate(indices):
    ex = train_data[idx]

    src = ex["source"]
    tgt = ex["target"]

    src_h, tgt_h = highlight_diff(src, tgt)

    html_output += f"""
    <div style="margin-bottom:18px;">
        <h4>[{i}] Sample Index: {idx}</h4>
        <div><strong> 원본 :</strong> {src_h}</div>
        <div><strong> 수정본 :</strong> {tgt_h}</div>
    </div>
    <hr>
    """

display(HTML(html_output))


📖 랜덤 상세 샘플 + 단어 하이라이트 (10개)


In [ ]:
# diff 함수: 단어 단위 + 문자 단위 하이라이트
def highlight_word_diff(src_word, tgt_word):
    if src_word == tgt_word:
        return src_word, tgt_word
    
    matcher = difflib.SequenceMatcher(None, src_word, tgt_word)
    
    src_h = ""
    tgt_h = ""
    
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        src_chunk = src_word[i1:i2]
        tgt_chunk = tgt_word[j1:j2]
        
        if tag == "equal":
            src_h += src_chunk
            tgt_h += tgt_chunk
        else:
            # 변경된 부분 하이라이트
            src_h += f"<b>{src_chunk}</b>"
            tgt_h += f"<b>{tgt_chunk}</b>"
    
    return src_h, tgt_h


def visualize_sentence_diff(src, tgt):
    src_words = src.split()
    tgt_words = tgt.split()

    # 길이가 다를 경우 zip 불가 → padding 처리
    max_len = max(len(src_words), len(tgt_words))
    src_words += [""] * (max_len - len(src_words))
    tgt_words += [""] * (max_len - len(tgt_words))

    src_res = []
    tgt_res = []

    for sw, tw in zip(src_words, tgt_words):
        h_src, h_tgt = highlight_word_diff(sw, tw)
        src_res.append(h_src)
        tgt_res.append(h_tgt)

    src_html = " ".join(src_res)
    tgt_html = " ".join(tgt_res)
    
    lev = levenshtein_distance(src, tgt)

    return src_html, tgt_html, lev


# -----------------------------
# 랜덤 샘플 5개 시각화
# -----------------------------
indices = random.sample(range(len(train_data)), 5)
html_output = ""

for i, idx in enumerate(indices):
    ex = train_data[idx]
    src = ex["source"]
    tgt = ex["target"]

    src_html, tgt_html, lev = visualize_sentence_diff(src, tgt)

    html_output += f"""
    <div style="margin-bottom:18px;">
        <h4>[{i}] Sample Index: {idx} | Levenshtein distance: {lev}</h4>
        <div><strong>Source (오류본):</strong> {src_html}</div>
        <div><strong>Target (수정본):</strong> {tgt_html}</div>
    </div>
    <hr>
    """

display(HTML(html_output))